In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\LTTS\New folder\medipin website\medical_website-main\dataset.csv.xls")
df = df.iloc[:412,:11]
df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
407,Impetigo,skin_rash,high_fever,blister,red_sore_around_nose,NaN,NaN,NaN,NaN,NaN,NaN
408,Impetigo,skin_rash,high_fever,blister,red_sore_around_nose,yellow_crust_ooze,NaN,NaN,NaN,NaN,NaN
409,Impetigo,skin_rash,blister,red_sore_around_nose,yellow_crust_ooze,NaN,NaN,NaN,NaN,NaN,NaN
410,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.isnull().sum()

Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Symptom_4      59
Symptom_5     123
Symptom_6     193
Symptom_7     234
Symptom_8     258
Symptom_9     280
Symptom_10    290
dtype: int64

In [4]:
# # Define a function to impute missing values with the mode within each group
# def impute_mode(group):
#     mode_values = group.mode()
#     if mode_values.empty:
#         return group
#     else:
#         mode_value = mode_values.iloc[0]
#         return group.fillna(mode_value)
# # Impute missing values in the 'category' column based on mode within each 'group'
# df['Symptom_4'] = df.groupby('Symptom_1')['Symptom_4'].apply(impute_mode)


Handeling Missing Data

In [5]:
df['Symptom_4'] = df.apply(lambda row: row['Symptom_4'] if pd.notna(row['Symptom_4']) else row.drop('Symptom_4').value_counts().sample(1, weights=row.drop('Symptom_4').value_counts().values).index[0], axis=1)

In [6]:
columns_to_fill_missing_values = ['Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10']
df[columns_to_fill_missing_values] = df.apply(lambda row: row[columns_to_fill_missing_values].apply(lambda x: x if pd.notna(x) else row.drop(columns_to_fill_missing_values).value_counts().sample(1, weights=row.drop(columns_to_fill_missing_values).value_counts().values).index[0]), axis=1)

In [7]:
df.isnull().sum()

Disease       0
Symptom_1     0
Symptom_2     0
Symptom_3     0
Symptom_4     0
Symptom_5     0
Symptom_6     0
Symptom_7     0
Symptom_8     0
Symptom_9     0
Symptom_10    0
dtype: int64

In [8]:
df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,Fungal infection,skin_rash,Fungal infection,itching,Fungal infection,Fungal infection
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,nodal_skin_eruptions,nodal_skin_eruptions,dischromic _patches,nodal_skin_eruptions,nodal_skin_eruptions,skin_rash,nodal_skin_eruptions
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,Fungal infection,Fungal infection,itching,dischromic _patches,dischromic _patches,Fungal infection,nodal_skin_eruptions
3,Fungal infection,itching,skin_rash,dischromic _patches,dischromic _patches,dischromic _patches,skin_rash,skin_rash,itching,dischromic _patches,dischromic _patches
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,itching,itching,itching,itching,skin_rash,itching,itching
...,...,...,...,...,...,...,...,...,...,...,...
407,Impetigo,skin_rash,high_fever,blister,red_sore_around_nose,high_fever,skin_rash,high_fever,skin_rash,high_fever,Impetigo
408,Impetigo,skin_rash,high_fever,blister,red_sore_around_nose,yellow_crust_ooze,blister,red_sore_around_nose,Impetigo,Impetigo,Impetigo
409,Impetigo,skin_rash,blister,red_sore_around_nose,yellow_crust_ooze,skin_rash,skin_rash,blister,skin_rash,blister,yellow_crust_ooze
410,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,skin_rash,dischromic _patches,skin_rash,itching,dischromic _patches,nodal_skin_eruptions


### Spliting the data

In [9]:
y = df['Disease']
df = df.drop('Disease',axis =1)

### Encoding

In [10]:
df.columns

Index(['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5',
       'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10'],
      dtype='object')

In [11]:
columns_to_encode = ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10',]
for column in columns_to_encode:
    one_hot = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, one_hot], axis=1)
    df = df.drop(column, axis=1)

In [12]:
one_hot = pd.get_dummies(y)
y = pd.concat([y, one_hot], axis=1)

In [13]:
df

,Symptom_1_ acidity,Symptom_1_ back_pain,Symptom_1_ bladder_discomfort,Symptom_1_ breathlessness,Symptom_1_ burning_micturition,Symptom_1_ chest_pain,Symptom_1_ chills,Symptom_1_ constipation,Symptom_1_ continuous_sneezing,Symptom_1_ cough,...,Symptom_10_Impetigo,Symptom_10_Jaundice,Symptom_10_Migraine,Symptom_10_Osteoarthristis,Symptom_10_Paralysis (brain hemorrhage),Symptom_10_Peptic ulcer diseae,Symptom_10_Psoriasis,Symptom_10_Urinary tract infection,Symptom_10_Varicose veins,Symptom_10_itching
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
408,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y = y.drop('Disease', axis =1)
y

,(vertigo) Paroymsal Positional Vertigo,AIDS,Acne,Alcoholic hepatitis,Allergy,Arthritis,Bronchial Asthma,Cervical spondylosis,Chicken pox,Chronic cholestasis,...,Osteoarthristis,Paralysis (brain hemorrhage),Peptic ulcer diseae,Pneumonia,Psoriasis,Tuberculosis,Typhoid,Urinary tract infection,Varicose veins,hepatitis A
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
x = df
x

,Symptom_1_ acidity,Symptom_1_ back_pain,Symptom_1_ bladder_discomfort,Symptom_1_ breathlessness,Symptom_1_ burning_micturition,Symptom_1_ chest_pain,Symptom_1_ chills,Symptom_1_ constipation,Symptom_1_ continuous_sneezing,Symptom_1_ cough,...,Symptom_10_Impetigo,Symptom_10_Jaundice,Symptom_10_Migraine,Symptom_10_Osteoarthristis,Symptom_10_Paralysis (brain hemorrhage),Symptom_10_Peptic ulcer diseae,Symptom_10_Psoriasis,Symptom_10_Urinary tract infection,Symptom_10_Varicose veins,Symptom_10_itching
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
408,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size = 0.2, random_state = 1)

In [17]:
# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42,max_features='log2')
classifier.fit(x_train, y_train)
y_train1=classifier.predict(x_train)
# accuracy_score(y_train, y_train1)

# # Making the Confusion Matrix
y_pred = classifier.predict(x_test)
# cm = ConfusionMatrixDisplay(y_test, y_pred)
# print(cm)
accuracy_score(y_test, y_pred)

0.7469879518072289